In [1]:
import requests
import pandas as pd
import time


SEARCH_TERM = "salads"

BASE_URL = "https://world.openfoodfacts.org/cgi/search.pl"

params = {
    "search_terms": SEARCH_TERM,
    "page_size": 50,
    "json": 1,
    "fields": "product_name,nutriments,brands,code,ingredients_text,serving_suggestions",
}

all_products = []
NUM_PAGES = 5

for page in range(1, NUM_PAGES + 1):
    success = False
    retries = 3
    for attempt in range(retries):
        try:
            print(f"🔄 Fetching page {page}, attempt {attempt + 1} for '{SEARCH_TERM}'...")
            params["page"] = page
            response = requests.get(BASE_URL, params=params, timeout=10)
            if response.status_code == 200:
                data = response.json()
                products = data.get("products", [])
                for product in products:
                    nutriments = product.get("nutriments", {})
                    all_products.append({
                        "Product Name": product.get("product_name", "N/A"),
                        "Brand": product.get("brands", "N/A"),
                        "Calories (kcal)": nutriments.get("energy-kcal_100g"),
                        "Proteins (g)": nutriments.get("proteins_100g"),
                        "Fat (g)": nutriments.get("fat_100g"),
                        "Sugar (g)": nutriments.get("sugars_100g"),
                        "Salt (g)": nutriments.get("salt_100g"),
                        "Carbohydrates (g)": nutriments.get("carbohydrates_100g"),
                        "Code": product.get("code"),
                        "Ingredients Text": product.get("ingredients_text", "N/A"),
                        "Serving Suggestions": product.get("serving_suggestions", "N/A")
                    })
                success = True
                break
            else:
                print(f"❌ Status code {response.status_code}")
        except Exception as e:
            print(f"⚠️ Error on attempt {attempt + 1}: {e}")
            time.sleep(2)

    if not success:
        print(f"❌ Failed to fetch page {page} after {retries} attempts.")
        break

    time.sleep(1)

# Save data
df = pd.DataFrame(all_products)
output_filename = f"openfoodfacts_{SEARCH_TERM}_data.csv"
df.to_csv(output_filename, index=False)

print(f"\n✅ Saved {len(df)} products to '{output_filename}'")

🔄 Fetching page 1, attempt 1 for 'salads'...
🔄 Fetching page 2, attempt 1 for 'salads'...
🔄 Fetching page 3, attempt 1 for 'salads'...
🔄 Fetching page 4, attempt 1 for 'salads'...
🔄 Fetching page 5, attempt 1 for 'salads'...

✅ Saved 250 products to 'openfoodfacts_salads_data.csv'
